In [1]:
from pathlib import Path
import subprocess
import os
from glob import glob
import numpy as np
import pandas as pd
from datetime import datetime
import joblib
import warnings
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.metrics import roc_auc_score
from tqdm.auto import tqdm
from sklearn.model_selection import TimeSeriesSplit, GroupKFold, StratifiedGroupKFold
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.datasets import load_wine
import pickle
import seaborn as sns

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("nllb/tokenize")

model = torch.load("nllb/nllb-200-1.3B.pt", weights_only = False)
model.to(device)
model.eval()

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-23): 24 x M2M100EncoderLayer(
          (self_attn): M2M100SdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=1024, bias=True)
   

In [3]:
import torch
print(torch.__version__)           # Should show 2.2.2+cu118
print(torch.cuda.is_available())   # Should return True

2.6.0+cu124
True


In [7]:
data_train = pd.read_json(path_or_buf="Data_MT/mt_train.jsonl", lines=True)
data_val = pd.read_json(path_or_buf="Data_MT/mt_dev.jsonl", lines=True)
data_test = pd.read_json(path_or_buf="Data_MT/mt_test.jsonl", lines=True)

In [8]:
data_train

,context,source,translation
0,patient : 癫痫发作后紧张想哭头痛发冷（男，30岁）😊 \ndoctor : 你好这...,我想咨询件事 我对别人有点敌意,ฉันอยากจะถามอะไรหน่อย ฉันรู้สึกไม่เป็นมิตรกับค...
1,patient : 妇科炎症会不会夫妻之间互相传播（女，30岁） \ndoctor : 你好...,我考虑细菌性或滴虫性阴道炎的可能性,ฉันกำลังคิดว่าน่าจะเกิดภาวะช่องคลอดอักเสบจากเช...
2,patient : 腰酸背痛，是不是骨质疏松啊！（女，47岁）颈椎挤压的头疼，恶心。图片因隐...,你好！腰痛发现有多长时间了？是腰中间疼痛还是两侧肌肉疼痛？,สวัสดี คุณมีอาการปวดหลังส่วนล่างมานานแค่ไหนแล้...
3,patient : 牙周炎后引起牙齿松动该怎么办？（女，30岁） \ndoctor : 你好...,是不是平时勤刷牙，多刷干净点就可以减少牙结石的沉积呢？,ถ้าแปรงฟันบ่อยๆ และทำความสะอาดบ่อยๆ ก็จะสามารถ...
4,patient : 慢性荨麻疹怎么根治？（女，20岁） \ndoctor : 你好，慢性荨麻...,周围环境的物理化学因素也可以引起身体过敏,ปัจจัยทางกายภาพและเคมีในสภาพแวดล้อมโดยรอบ ก็สา...
...,...,...,...
18595,patient : 肚脐眼里疼是不是南胃炎（男，49岁） \ndoctor : 您好，这种情...,[自动回复]您好，现在比较忙，请您留言具体问题，稍后我会及时回复您的问题，请谅解。,[ตอบกลับอัตโนมัติ] สวัสดี ตอนนี้ฉันค่อนข้างยุ่...
18596,patient : 孕30周有点儿感冒可以喝姜汤红糖水吗（女，21岁） \ndoctor :...,[自动回复]您好，现在比较忙，请您留言具体问题，稍后我会及时回复您的问题，请谅解。,[ตอบกลับอัตโนมัติ] สวัสดี ตอนนี้หมอค่อนข้างยุ่...
18597,patient : 急性支气管炎入院，右肺多方占位（男，44）图片因隐私问题无法显示 \nd...,[自动回复]您好，现在比较忙，请您留言具体问题，稍后我会及时回复您的问题，请谅解。,[ตอบกลับอัตโนมัติ] สวัสดี ตอนนี้หมอค่อนข้างยุ่...
18598,patient : 医生您好，这是我的问题：月经不调（女，34岁） \ndoctor : 您...,[自动回复]您好，现在比较忙，请您留言具体问题，稍后我会及时回复您的问题，请谅解。,[ตอบกลับอัตโนมัติ] สวัสดี ตอนนี้ฉันค่อนข้างยุ่...


In [9]:
from datasets import Dataset, DatasetDict
from transformers import (
    NllbTokenizer, 
    M2M100ForConditionalGeneration,
    Trainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType
)
from torch.utils.data import DataLoader
import numpy as np
from sklearn.model_selection import train_test_split
import logging
from IPython.display import display, HTML
import matplotlib.pyplot as plt

class Config:
    SRC_LANG = "zho_Hans"  # Simplified Chinese
    TGT_LANG = "tha_Thai"  # Thai
    
    # Training parameters
    OUTPUT_DIR = "./chinese-thai-translation-model"
    NUM_EPOCHS = 4
    BATCH_SIZE = 2  # Adjust based on your GPU memory
    LEARNING_RATE = 3e-5
    MAX_LENGTH = 512

config = Config()

In [8]:
datasets = DatasetDict({
    'train': Dataset.from_dict({'source': data_train['source'], 'translation': data_train['translation']}),
    'validation': Dataset.from_dict({'source': data_val['source'], 'translation': data_val['translation']}),
})

In [9]:
datasets

DatasetDict({
    train: Dataset({
        features: ['source', 'translation'],
        num_rows: 18600
    })
    validation: Dataset({
        features: ['source', 'translation'],
        num_rows: 3000
    })
})

In [10]:
tokenizer.src_lang = config.SRC_LANG
tokenizer.tgt_lang = config.TGT_LANG

In [11]:
def tokenize_function(examples):
    # Tokenize Chinese (source)
    inputs = tokenizer(
        examples['source'], 
        truncation=True, 
        padding=False, 
        max_length=config.MAX_LENGTH
    )
    
    # Tokenize Thai (target)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples['translation'], 
            truncation=True, 
            padding=False, 
            max_length=config.MAX_LENGTH
        )
    
    inputs['labels'] = labels['input_ids']
    return inputs

tokenized_datasets = datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=datasets['train'].column_names
)

Map:   0%|          | 0/18600 [00:00<?, ? examples/s]

/home/tb1017/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [12]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 18600
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3000
    })
})

In [13]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True
)

In [14]:
training_args = TrainingArguments(
    output_dir=config.OUTPUT_DIR,
    num_train_epochs=config.NUM_EPOCHS,
    per_device_train_batch_size=config.BATCH_SIZE,
    per_device_eval_batch_size=config.BATCH_SIZE,
    gradient_accumulation_steps=4,
    warmup_steps=1000,
    learning_rate=config.LEARNING_RATE,
    weight_decay=0.01,
    logging_dir=f"{config.OUTPUT_DIR}/logs",
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=torch.cuda.is_available(),
    report_to="tensorboard",
    remove_unused_columns=True,                             # Saves memory; set False if using custom dataset fields
    dataloader_pin_memory=True,                             # Helpful on some systems
    gradient_checkpointing=True,
    dataloader_num_workers=2,                               # Speeds up data loading; adjust as needed
    optim="adamw_torch",
)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

/tmp/ipykernel_27639/234206557.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:
def print_trainable_parameters(model):
    trainable = 0
    total = 0
    for param in model.parameters():
        total += param.numel()
        if param.requires_grad:
            trainable += param.numel()
    print(f"Trainable parameters: {trainable:,}")
    print(f"Total parameters:     {total:,}")
    print(f"Trainable %:          {100 * trainable / total:.2f}%")

print_trainable_parameters(model)

Trainable parameters: 615,073,792
Total parameters:     615,073,792
Trainable %:          100.00%


In [ ]:
trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss


In [55]:
modelx = trainer.model.to("cuda")

In [27]:
model

In [56]:
import json
def translate_text(chinese_text, max_length=256):
    # Tokenize input
    inputs = tokenizer(chinese_text, return_tensors="pt", padding=True).to("cuda")
    
    # Move to GPU if available
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
        modelx.cuda()
        
    # Generate translation
    with torch.no_grad():
        generated_tokens = modelx.generate(
            **inputs,
            forced_bos_token_id=256175,
        )
    
    # Decode the translation
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return translation

In [60]:
translate_text(data_val['source'][0])

'อาการบาดเจ็บที่สมองทําให้มองเห็นไม่ชัดและไม่สม่ําเสมอ (ชาย อายุ 40 ปี)'

In [57]:
import tqdm
predictions = []
for text in tqdm.tqdm(data_val['source']):
    predictions.append(translate_text(text))

predictions

100%|██████████| 3000/3000 [18:05<00:00,  2.76it/s]


['อาการบาดเจ็บที่สมองทําให้สายตาไม่ชัดและไม่สติ (ชาย อายุ 40 ปี)',
 'เดือนที่แล้วประจําเดือนครั้งแรกของฉันเริ่มเดือน และครั้งที่สองมาวันที่ 22 และสิ้นสุด',
 'บางคนรู้สึกไว ทําไม่สบาย จะปวดและบวม',
 'ความเครียดมาก การไม่พิจารณาเรื่องการดื่ม การกินจะทําให้เกิดภาวะโลหิตจางลดน้ําตาลหรือไม่ หมอซุน',
 'ตอนนี้ฉันรู้สึกเหมือนคนทั้งตัวจะฝันหลอน',
 'เลือดออกนี้ไม่เกี่ยวข้องกับกระดูกที่แพร่กระจาย แต่กลัวว่าเนื้องอกจะทําลายหลอดเลือดและมีเลือดออก',
 'หากเป็นโรคลมชักจากการติดเชื้อ หรือเป็นโรคลมชักจากเลือดละลาย หรือโรคลมชักสูงมาก อาจเป็นได้',
 'ในกรณีของฉัน คุณแนะนําให้ฉันกินยาอะไรดี?',
 'ด้วยความยินดี ตอนนี้มีคําถามเพิ่มเติม! หากคุณรู้สึกว่าคําตอบมีประโยชน์ โปรดคลิก "พอใจมาก" เพื่อประเมินความพึงพอใจของคุณ และสนับสนุนฉัน อย่ากังวลว่า "พอใจมาก" หรือ "พอใจมาก" กติกาของโปรแกรมนี้เป็นพิเศษมาก พอใจก็จะโทษฉันด้วย 😂 - -',
 'ลูกฉันโกรธมาก! แต่ 5 วันแล้ว และยังเป็นอีก',
 'ฉันตรวจทุกครั้งแล้วตรวจแล้วหมดแล้ว แม่ฉันก็กลัวที่จะส่องกล้องลําไส้ใหญ่ ฉันไม่มีทางทํา',
 'อืม อันนี้ร้ายแรงไหม? ในอนาคตจะกําเริบอีกไหม?',


In [58]:
ref = []
ref = data_val['translation']
ref

0       ตาพร่ามัวและหมดสติเนื่องจากสมองบาดเจ็บ (ชายอาย...
1       เดือนที่แล้ว ประจำเดือนครั้งแรกมาตอนต้นเดือน ค...
2                  บางคนแพ้ ทำแล้วไม่สบาย อึดอัดและปวดบวม
3       ความเครียดเยอะ เบื่ออาหารและน้ำ จะทำให้ผอมลงแล...
4              ความรู้สึกตอนนี้คือเหมือนมีอาการประสาทหลอน
                              ...                        
2995    สวัสดี หินปูนเป็นรอยโรคที่หลงเหลืออยู่หลังจากก...
2996            พี่สาวของฉันบอกให้กิน เธอเป็นสูตินรีแพทย์
2997          วิธีการรักษาโรคประสาทอ่อน? (ชาย อายุ 31 ปี)
2998    หลังการสัมผัสเชื้อมา 3 เดือนสามารถยืนยันได้อย่...
2999    ข้อใดข้อหนึ่งก็สามารถตัดความเสี่ยงของการติดเชื...
Name: translation, Length: 3000, dtype: object

In [60]:
from pythainlp.tokenize import word_tokenize
import sacrebleu
# Tokenize predictions and references
tokenized_preds = [" ".join(word_tokenize(pred)) for pred in predictions]
tokenized_refs = [" ".join(word_tokenize(r)) for r in ref]

# Compute BLEU with no tokenization (since we already tokenized)
score = sacrebleu.corpus_bleu(tokenized_preds, [tokenized_refs], tokenize="none")
print(f"BLEU score: {score.score:.2f}")

BLEU score: 25.60


In [ ]:
answer = []
for text in tqdm.tqdm(data_test['source']):
    answer.append(translate_text(text))

In [ ]:
with open("summit_MT/nllb-200-dist-600M(fine-tune-optimize).txt", "w", encoding="utf-8") as f:
    for item in answer:
        f.write(f"{item}\n")